# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/v1/rerank`(cross encoder rerank model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-07-23 20:34:37] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=32460, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=344178870, constrained_json_whitespace_pattern=None, w

[2025-07-23 20:34:54] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-07-23 20:34:54] Init torch distributed begin.


[2025-07-23 20:34:55] Init torch distributed ends. mem usage=0.00 GB


[2025-07-23 20:34:56] Load weight begin. avail mem=76.30 GB


[2025-07-23 20:34:57] Using model weights format ['*.safetensors']


...

[2025-07-23 20:34:59] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=50.21 GB


[2025-07-23 20:34:59] INFO:     Started server process [2861283]
[2025-07-23 20:34:59] INFO:     Waiting for application startup.
[2025-07-23 20:34:59] INFO:     Application startup complete.
[2025-07-23 20:34:59] INFO:     Uvicorn running on http://0.0.0.0:32460 (Press CTRL+C to quit)


[2025-07-23 20:35:00] INFO:     127.0.0.1:38632 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-23 20:35:00] INFO:     127.0.0.1:38648 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-23 20:35:00] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-23 20:35:01] INFO:     127.0.0.1:38660 - "POST /generate HTTP/1.1" 200 OK
[2025-07-23 20:35:01] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-23 20:35:05] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-23 20:35:05] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.14, #queue-req: 0, 


[2025-07-23 20:35:05] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 125.46, #queue-req: 0, 


[2025-07-23 20:35:06] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 125.95, #queue-req: 0, 
[2025-07-23 20:35:06] INFO:     127.0.0.1:34490 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.
- `preferred_sampling_params`: The default sampling params specified via `--preferred-sampling-params`. `None` is returned in this example as we did not explicitly configure it in server args.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["preferred_sampling_params"] is None
assert response_json.keys() == {
    "model_path",
    "is_generation",
    "tokenizer_path",
    "preferred_sampling_params",
}

[2025-07-23 20:35:06] INFO:     127.0.0.1:34504 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-07-23 20:35:06] INFO:     127.0.0.1:34520 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-07-23 20:35:06] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-23 20:35:07] INFO:     127.0.0.1:34536 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-07-23 20:35:07] INFO:     127.0.0.1:34538 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-07-23 20:35:07] Cache flushed successfully!
[2025-07-23 20:35:07] INFO:     127.0.0.1:34548 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-07-23 20:35:07] Start update_weights. Load format=auto
[2025-07-23 20:35:07] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-07-23 20:35:07] Using model weights format ['*.safetensors']
[2025-07-23 20:35:07] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.85it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.85it/s]

[2025-07-23 20:35:07] Update weights end.
[2025-07-23 20:35:07] Cache flushed successfully!
[2025-07-23 20:35:07] INFO:     127.0.0.1:34564 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-07-23 20:35:07] Start update_weights. Load format=auto
[2025-07-23 20:35:07] Update engine weights online from disk begin. avail mem=41.46 GB
[2025-07-23 20:35:07] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-07-23 20:35:07] INFO:     127.0.0.1:34578 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-07-23 20:35:07] Child process unexpectedly failed with exitcode=9. pid=2862275
[2025-07-23 20:35:07] Child process unexpectedly failed with exitcode=9. pid=2861737


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-23 20:35:18] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=38593, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=567787072, constrained_json_whitespac

[2025-07-23 20:35:30] Downcasting torch.float32 to torch.float16.


[2025-07-23 20:35:30] Overlap scheduler is disabled for embedding models.
[2025-07-23 20:35:30] Downcasting torch.float32 to torch.float16.
[2025-07-23 20:35:30] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-07-23 20:35:30] Init torch distributed begin.


[2025-07-23 20:35:31] Init torch distributed ends. mem usage=0.00 GB


[2025-07-23 20:35:32] Load weight begin. avail mem=62.05 GB


...

[2025-07-23 20:35:37] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-07-23 20:35:37] Memory pool end. avail mem=41.18 GB


[2025-07-23 20:35:37] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=27.09 GB


[2025-07-23 20:35:38] INFO:     Started server process [2864342]
[2025-07-23 20:35:38] INFO:     Waiting for application startup.
[2025-07-23 20:35:38] INFO:     Application startup complete.
[2025-07-23 20:35:38] INFO:     Uvicorn running on http://0.0.0.0:38593 (Press CTRL+C to quit)


[2025-07-23 20:35:38] INFO:     127.0.0.1:59288 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-23 20:35:39] INFO:     127.0.0.1:59290 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-23 20:35:39] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-23 20:35:39] INFO:     127.0.0.1:59292 - "POST /encode HTTP/1.1" 200 OK
[2025-07-23 20:35:39] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-07-23 20:35:44] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-07-23 20:35:44] INFO:     127.0.0.1:49502 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-07-23 20:35:44] Child process unexpectedly failed with exitcode=9. pid=2864924
[2025-07-23 20:35:44] Child process unexpectedly failed with exitcode=9. pid=2864621


## v1/rerank (cross encoder rerank model)
Rerank a list of documents given a query using a cross-encoder model. Note that this API is only available for cross encoder model like [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) with `attention-backend` `triton` and `torch_native`.


In [14]:
reranker_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path BAAI/bge-reranker-v2-m3 \
    --host 0.0.0.0 --disable-radix-cache --chunked-prefill-size -1 --attention-backend triton --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-23 20:35:54] server_args=ServerArgs(model_path='BAAI/bge-reranker-v2-m3', tokenizer_path='BAAI/bge-reranker-v2-m3', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=35001, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=951935288, constrained_json_whitespace_pattern=None, watchdog_t

[2025-07-23 20:36:06] Downcasting torch.float32 to torch.float16.


[2025-07-23 20:36:06] Overlap scheduler is disabled for embedding models.
[2025-07-23 20:36:06] Downcasting torch.float32 to torch.float16.
[2025-07-23 20:36:06] Init torch distributed begin.


[2025-07-23 20:36:07] Init torch distributed ends. mem usage=0.00 GB


[2025-07-23 20:36:08] Load weight begin. avail mem=46.54 GB
[2025-07-23 20:36:08] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.66it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.66it/s]

[2025-07-23 20:36:09] Load weight end. type=XLMRobertaForSequenceClassification, dtype=torch.float16, avail mem=45.44 GB, mem usage=1.10 GB.
[2025-07-23 20:36:09] KV Cache is allocated. #tokens: 20480, K size: 0.94 GB, V size: 0.94 GB
[2025-07-23 20:36:09] Memory pool end. avail mem=43.40 GB


[2025-07-23 20:36:10] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8194, available_gpu_mem=43.30 GB


[2025-07-23 20:36:11] INFO:     Started server process [2867386]
[2025-07-23 20:36:11] INFO:     Waiting for application startup.
[2025-07-23 20:36:11] INFO:     Application startup complete.
[2025-07-23 20:36:11] INFO:     Uvicorn running on http://0.0.0.0:35001 (Press CTRL+C to quit)


[2025-07-23 20:36:12] INFO:     127.0.0.1:38944 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-23 20:36:12] INFO:     127.0.0.1:38950 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-23 20:36:12] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-23 20:36:12] INFO:     127.0.0.1:38966 - "POST /encode HTTP/1.1" 200 OK
[2025-07-23 20:36:12] The server is fired up and ready to roll!


In [15]:
# compute rerank scores for query and documents

url = f"http://localhost:{port}/v1/rerank"
data = {
    "model": "BAAI/bge-reranker-v2-m3",
    "query": "what is panda?",
    "documents": [
        "hi",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
}

response = requests.post(url, json=data)
response_json = response.json()
for item in response_json:
    print_highlight(f"Score: {item['score']:.2f} - Document: '{item['document']}'")

[2025-07-23 20:36:17] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-07-23 20:36:17] Prefill batch. #new-seq: 1, #new-token: 43, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-07-23 20:36:17] INFO:     127.0.0.1:56116 - "POST /v1/rerank HTTP/1.1" 200 OK


In [16]:
terminate_process(reranker_process)

[2025-07-23 20:36:17] Child process unexpectedly failed with exitcode=9. pid=2867889
[2025-07-23 20:36:17] Child process unexpectedly failed with exitcode=9. pid=2867621


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [17]:
# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-23 20:36:27] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30743, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1013209763, constrained_jso

[2025-07-23 20:36:39] Overlap scheduler is disabled for embedding models.
[2025-07-23 20:36:39] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-23 20:36:39] Init torch distributed begin.


[2025-07-23 20:36:39] Init torch distributed ends. mem usage=0.00 GB


[2025-07-23 20:36:41] Load weight begin. avail mem=74.50 GB


[2025-07-23 20:36:41] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.09it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.64it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.40it/s]

[2025-07-23 20:36:44] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=60.46 GB, mem usage=14.04 GB.
[2025-07-23 20:36:44] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-07-23 20:36:44] Memory pool end. avail mem=57.66 GB


[2025-07-23 20:36:45] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.06 GB


[2025-07-23 20:36:45] INFO:     Started server process [2870235]
[2025-07-23 20:36:45] INFO:     Waiting for application startup.
[2025-07-23 20:36:45] INFO:     Application startup complete.
[2025-07-23 20:36:45] INFO:     Uvicorn running on http://0.0.0.0:30743 (Press CTRL+C to quit)


[2025-07-23 20:36:46] INFO:     127.0.0.1:39874 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-23 20:36:46] INFO:     127.0.0.1:39888 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-23 20:36:46] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [18]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-07-23 20:37:53] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-07-23 20:37:53] INFO:     127.0.0.1:39890 - "POST /encode HTTP/1.1" 200 OK
[2025-07-23 20:37:53] The server is fired up and ready to roll!


[2025-07-23 20:38:07] INFO:     127.0.0.1:51526 - "POST /classify HTTP/1.1" 200 OK


In [19]:
terminate_process(reward_process)

[2025-07-23 20:38:07] Child process unexpectedly failed with exitcode=9. pid=2871005


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [20]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-07-23 20:38:18] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=35967, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=497671007, constrained_json_whitespace_pattern=None, watchdog_

[2025-07-23 20:38:29] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-23 20:38:29] Init torch distributed begin.


[2025-07-23 20:38:30] Init torch distributed ends. mem usage=0.00 GB


[2025-07-23 20:38:32] Load weight begin. avail mem=58.35 GB
[2025-07-23 20:38:32] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-07-23 20:38:38] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=45.96 GB


[2025-07-23 20:38:39] INFO:     Started server process [2877420]
[2025-07-23 20:38:39] INFO:     Waiting for application startup.
[2025-07-23 20:38:39] INFO:     Application startup complete.
[2025-07-23 20:38:39] INFO:     Uvicorn running on http://0.0.0.0:35967 (Press CTRL+C to quit)


[2025-07-23 20:38:40] INFO:     127.0.0.1:53304 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-23 20:38:40] INFO:     127.0.0.1:53320 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-23 20:38:40] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-23 20:38:41] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1d-gpu-45/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [21]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-07-23 20:38:55] Resetting ExpertDistributionRecorder...
[2025-07-23 20:38:55] INFO:     127.0.0.1:55700 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-23 20:38:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-07-23 20:38:55] INFO:     127.0.0.1:53326 - "POST /generate HTTP/1.1" 200 OK
[2025-07-23 20:38:55] The server is fired up and ready to roll!


[2025-07-23 20:38:56] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, cuda graph: False, gen throughput (token/s): 2.62, #queue-req: 0, 


[2025-07-23 20:38:57] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, cuda graph: False, gen throughput (token/s): 38.58, #queue-req: 0, 


[2025-07-23 20:38:58] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, cuda graph: False, gen throughput (token/s): 38.75, #queue-req: 0, 


[2025-07-23 20:38:58] INFO:     127.0.0.1:48534 - "POST /generate HTTP/1.1" 200 OK


[2025-07-23 20:38:58] INFO:     127.0.0.1:48542 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-23 20:39:00] Write expert distribution to /tmp/expert_distribution_recorder_1753303140.2945251.pt
[2025-07-23 20:39:00] Resetting ExpertDistributionRecorder...
[2025-07-23 20:39:00] INFO:     127.0.0.1:48556 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [22]:
terminate_process(expert_record_server_process)

[2025-07-23 20:39:00] Child process unexpectedly failed with exitcode=9. pid=2877961
[2025-07-23 20:39:00] Child process unexpectedly failed with exitcode=9. pid=2877745
